In [1]:
import numpy as np
import tensorflow as tf

E:\06_software\Anoconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 自定义损失函数并求解

In [16]:
# 两个输入节点
x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
# 回归问题 一般只有一个输出节点
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

# 定义一个单层的神经网络前向传播的过程
w1 = tf.Variable(tf.random_normal([2,1], stddev=1, seed=1))
y = tf.matmul(x, w1)

# 自定义损失函数
loss_less = 10  # 预测少的的成本
loss_more = 1  # 预测多的成本
loss = tf.reduce_mean(tf.where(tf.greater(y, y_), (y - y_) * loss_more, (y_-y) * loss_less))

# 用Adam求解损失函数最小值
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# 随机生成一个模拟数据集
rdm = np.random.RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [x1 + x2 + rdm.rand() / 10 - 0.05 for (x1, x2) in X]
Y = np.array(Y).reshape(-1, 1)  # 转换成与y相同的形状

# 训练神经网络
batch_size = 8
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        sess.run(train_step, feed_dict={x: X[start: end], y_: Y[start: end]})
    print(sess.run(w1))

[[1.019347 ]
 [1.0428089]]


In [31]:
def get_weight(shape, lamda):
    """
    获取一层神经边上的权重 并将这个权重的L2正则化 损失加入名称为losses的集合中
    """
    var = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lamda)(var))
    return var

x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
batch_size = 8

# 定义每一层网络中神经元个数
layer_dimension = [2, 10, 10, 10, 1]
# 神经网络的层数
n_layers = len(layer_dimension)

# 这个变量维护前向传播时最深层的节点 开始的时候就是输入层
cur_layer = x
# 当前层神经元个数
in_dimension = layer_dimension[0]

# 通过一个循环来生成5层全连接的神经网络结构
for i in range(1, n_layers):
    out_dimension = layer_dimension[i] # 下一层神经元个数
    weight = get_weight([in_dimension, out_dimension], 0.001)
    bias = tf.Variable(tf.constant(0.1, shape=[out_dimension]))
    cur_layer = tf.nn.relu(tf.matmul(cur_layer, weight) + bias)
    # 进入下一层之前将下一层神经元个数更新为当前层神经元个数
    in_dimension = layer_dimension[i]

# 在定义神经网络前向传播的同时已经将所有的L2正则化损失加入了图上的集合
# 这里仅需计算刻画模型在训练集上表现的损失函数
mse_loss = tf.reduce_mean(tf.square(y_ - cur_layer))

# 将均方误差损失函数加入到损失集合
tf.add_to_collection('losses', mse_loss)

# get_collection返回一个列表 这个列表是所有这个集合中的元素
loss = tf.add_n(tf.get_collection('losses'))

# 用Adam求解损失函数最小值
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# 随机生成一个模拟数据集
rdm = np.random.RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [x1 + x2 + rdm.rand() / 10 - 0.05 for (x1, x2) in X]
Y = np.array(Y).reshape(-1, 1)  # 转换成与y相同的形状


tf.reset_default_graph()
# 训练神经网络
batch_size = 8
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        sess.run(train_step, feed_dict={x: X[start: end], y_: Y[start: end]})
    print(sess.run(weight))

[[ 0.58651805]
 [ 0.98028255]
 [ 0.31031385]
 [ 0.48067787]
 [-1.30291   ]
 [ 0.32806247]
 [ 0.3879062 ]
 [ 0.0951909 ]
 [-0.06792744]
 [-0.5234168 ]]


In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(r"C:\Users\youlin\Desktop", one_hot=True)

INPUT_NODE = 784
OUTPUT_NODE = 10

LAYER1_NODE = 500
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DEACY = 0.99

def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    """
    计算神经网络前向传播结果
    """
    if avg_class == None:  # 没有提供滑动平均类时 直接使用参数当前的取值
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return tf.matmul(layer1, weights2) + biases2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], 'y-input')
    
    # 生成隐层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    
    # 生成输出层的参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    # 计算在当前参数下神经网络前向传播的结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义存储训练轮数的变量 这个变量不需要训练 所以指定为trainable=False
    global_step = tf.Variable(0, trainable=False)
    
    # 给定滑动平均衰减率和训练轮数变量 初始化滑动平均数
    variables_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DEACY, global_step)
    
    # 在所有代表神经网络参数的变量上使用滑动平均
    variables_averages_op = variables_averages.apply(tf.trainable_variables())
    
    # 计算使用了滑动平均之后的前向传播结果
    average_y = inference(x, variables_averages, weights1, biases1, weights2, biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.arg_max(y_, 1))
    
    # 计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1) + regularizer(weights2)
    
    # 计算总损失
    loss = cross_entropy_mean + regularization
    
    # 设置指数衰减学习率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, 
                                               global_step, 
                                               mnist.train.num_examples / BATCH_SIZE,
                                              LEARNING_RATE_DECAY)
    
    # 选择优化方法
    train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
    
    # 在训练神经网络时 每过一遍数据既需要通过反向传播来更新神经网络的参数 又要更新每一个参数的滑动平均值
    # 为了一次完成多个操作 可以使用tf.group
    train_op = tf.group(train_step, variables_averages_op)
    
    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 准备验证数据 一般在神经网络的训练过程中会通过验证数据来大致判断停止的条件和评判训练的效果
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        
        # 准备测试数据 在生产环境中 这部分数据在训练时是不可见的 这个数据只是作为模型优劣的最后评价标准
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}
        
        # 迭代地训练神经网络
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:  # 每1000轮输出一次在验证集上的测试结果
                # 由于验证集较小 没有分batch
                validate_auc = sess.run(accuracy, feed_dict=validate_feed)
                print("After {} training step(s), validation accuracy using average model is {}".format(i, validate_auc))
            # 产生当前迭代的batch 并训练
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})
        
        # 在训练结束之后 在测试集上检测神经网络模型的最终正确率
        test_auc = sess.run(accuracy, feed_dict=test_feed)
        print("After {} training step(s), test accuracy using average model is {}".format(TRAINING_STEPS, test_auc))
    
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets(r"C:\Users\youlin\Desktop", one_hot=True)
    train(mnist)
    

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting C:\Users\youlin\Desktop\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting C:\Users\youlin\Desktop\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting C:\Users\youlin\Desktop\t10k-images-idx3-ubyte.gz
Extracting C:\Users\youlin\Desktop\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [11]:
main()

Extracting C:\Users\youlin\Desktop\train-images-idx3-ubyte.gz
Extracting C:\Users\youlin\Desktop\train-labels-idx1-ubyte.gz
Extracting C:\Users\youlin\Desktop\t10k-images-idx3-ubyte.gz
Extracting C:\Users\youlin\Desktop\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `argmax` instead
After 0 training step(s), validation accuracy using average model is 0.1509999930858612
After 1000 training step(s), validation accuracy using average model is 0.9760000109672546
After 2000 training step(s), validation accuracy using average model is 0.9825999736785889
After 3000 training step(s), validation accuracy using average model is 0.9825999736785889
After 4000 training step(s), validation accuracy using average model is 0.984000027179718
After 5000 training step(s), validation accuracy using average model is 0.9846000075340271
After 6000 training step(s), validation accuracy using average model is 0.9860000014305115
After 7000 training step(s), validation accuracy using average model is 0

In [44]:
sess = tf.InteractiveSession()
sess.run(tf.argmax(mnist.test.labels,1))

E:\06_software\Anoconda\lib\site-packages\tensorflow\python\client\session.py:1662: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [15]:
def inference(input_tensor, avg_class, reuse=False):
    """
    定义前向传播过程
    """
    # 定义第一层神经网络的变量和前向传播过程
    with tf.variable_scope('layer1', reuse=reuse):
        weights = tf.get_variable("weights", [INPUT_NODE, LAYER1_NODE], initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        if avg_class:
            weights = avg_class.average(weights)
            biases = avg_class.average(biases)
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
    
    # 类似地定义第二层神经网络的变量和前向传播过程
    with tf.variable_scope('layer2', reuse=reuse):
        weights = tf.get_variable("weights", [LAYER1_NODE, OUTPUT_NODE], initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        if avg_class:
            weights = avg_class.average(weights)
            biases = avg_class.average(biases)       
        layer2 = tf.matmul(input_tensor, weights) + biases
    # 返回前向传播结果
    return layer2

In [20]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(r"C:\Users\youlin\Desktop", one_hot=True)

INPUT_NODE = 784
OUTPUT_NODE = 10

LAYER1_NODE = 500
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DEACY = 0.99
tf.reset_default_graph()

def inference(input_tensor, avg_class, reuse=False):
    """
    定义前向传播过程
    """
    # 定义第一层神经网络的变量和前向传播过程
    with tf.variable_scope('layer1', reuse=reuse):
        weights = tf.get_variable("weights", [INPUT_NODE, LAYER1_NODE], initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        if avg_class:
            weights = avg_class.average(weights)
            biases = avg_class.average(biases)
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
    
    # 类似地定义第二层神经网络的变量和前向传播过程
    with tf.variable_scope('layer2', reuse=reuse):
        weights = tf.get_variable("weights", [LAYER1_NODE, OUTPUT_NODE], initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        if avg_class:
            weights = avg_class.average(weights)
            biases = avg_class.average(biases)       
        layer2 = tf.matmul(layer1, weights) + biases
    # 返回前向传播结果
    return layer2


# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], 'y-input')
    
    # 计算在当前参数下神经网络前向传播的结果
    y = inference(x, None)
    
    # 定义存储训练轮数的变量 这个变量不需要训练 所以指定为trainable=False
    global_step = tf.Variable(0, trainable=False)
    
    # 给定滑动平均衰减率和训练轮数变量 初始化滑动平均数
    variables_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DEACY, global_step)
    
    # 在所有代表神经网络参数的变量上使用滑动平均
    variables_averages_op = variables_averages.apply(tf.trainable_variables())
    
    # 计算使用了滑动平均之后的前向传播结果
    average_y = inference(x, variables_averages, True)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.arg_max(y_, 1))
    
    # 计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    with tf.variable_scope("", reuse=True):
        regularization = regularizer(tf.get_variable("layer1/weights")) + regularizer(tf.get_variable("layer2/weights"))
    
    # 计算总损失
    loss = cross_entropy_mean + regularization
    
    # 设置指数衰减学习率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, 
                                               global_step, 
                                               mnist.train.num_examples / BATCH_SIZE,
                                              LEARNING_RATE_DECAY)
    
    # 选择优化方法
    train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
    
    # 在训练神经网络时 每过一遍数据既需要通过反向传播来更新神经网络的参数 又要更新每一个参数的滑动平均值
    # 为了一次完成多个操作 可以使用tf.group
    train_op = tf.group(train_step, variables_averages_op)
    
    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 准备验证数据 一般在神经网络的训练过程中会通过验证数据来大致判断停止的条件和评判训练的效果
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        
        # 准备测试数据 在生产环境中 这部分数据在训练时是不可见的 这个数据只是作为模型优劣的最后评价标准
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}
        
        # 迭代地训练神经网络
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:  # 每1000轮输出一次在验证集上的测试结果
                # 由于验证集较小 没有分batch
                validate_auc = sess.run(accuracy, feed_dict=validate_feed)
                print("After {} training step(s), validation accuracy using average model is {}".format(i, validate_auc))
            # 产生当前迭代的batch 并训练
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})
        
        # 在训练结束之后 在测试集上检测神经网络模型的最终正确率
        test_auc = sess.run(accuracy, feed_dict=test_feed)
        print("After {} training step(s), test accuracy using average model is {}".format(TRAINING_STEPS, test_auc))
    
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets(r"C:\Users\youlin\Desktop", one_hot=True)
    train(mnist)
    

Extracting C:\Users\youlin\Desktop\train-images-idx3-ubyte.gz
Extracting C:\Users\youlin\Desktop\train-labels-idx1-ubyte.gz
Extracting C:\Users\youlin\Desktop\t10k-images-idx3-ubyte.gz
Extracting C:\Users\youlin\Desktop\t10k-labels-idx1-ubyte.gz


In [21]:
main()

Extracting C:\Users\youlin\Desktop\train-images-idx3-ubyte.gz
Extracting C:\Users\youlin\Desktop\train-labels-idx1-ubyte.gz
Extracting C:\Users\youlin\Desktop\t10k-images-idx3-ubyte.gz
Extracting C:\Users\youlin\Desktop\t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.07280000299215317
After 1000 training step(s), validation accuracy using average model is 0.9775999784469604
After 2000 training step(s), validation accuracy using average model is 0.9811999797821045
After 3000 training step(s), validation accuracy using average model is 0.9825999736785889
After 4000 training step(s), validation accuracy using average model is 0.9847999811172485
After 5000 training step(s), validation accuracy using average model is 0.9833999872207642
After 6000 training step(s), validation accuracy using average model is 0.9842000007629395
After 7000 training step(s), validation accuracy using average model is 0.9837999939918518
After 8000 training step(s),

In [17]:
with tf.variable_scope("", reuse=True):
    print(tf.get_variable("temp1/v1"))

<tf.Variable 'temp1/v1:0' shape=(4,) dtype=float32_ref>


In [7]:
sess = tf.InteractiveSession()
with tf.variable_scope('temp1'):
    v1 = tf.get_variable("v1", [4], initializer=tf.constant_initializer(3.0))
print(tf.get_variable("temp1/v1"))

E:\06_software\Anoconda\lib\site-packages\tensorflow\python\client\session.py:1662: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


ValueError: Variable temp1/v1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "E:\06_software\Anoconda\lib\site-packages\tensorflow\python\framework\ops.py", line 1768, in __init__
    self._traceback = tf_stack.extract_stack()
  File "E:\06_software\Anoconda\lib\site-packages\tensorflow\python\framework\ops.py", line 3272, in create_op
    op_def=op_def)
  File "E:\06_software\Anoconda\lib\site-packages\tensorflow\python\util\deprecation.py", line 488, in new_func
    return func(*args, **kwargs)
